# Clustering RFM
This notebook performs clustering on the Instacart Dataset to segment users based on the Recency, Frequency and Monetary values

## Clustering the data for customer segmentation

In [ ]:
#loading the necessary packages 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer

%matplotlib inline 
from matplotlib import rc

In [ ]:
# Define a function to test KMeans at various k
# This approach uses silhouette score to evaluate KMeans
def optimal_kmeans(dataset, start=2, end=11):
    '''
    Calculate the optimal number of kmeans
    
    INPUT:
        dataset : dataframe. Dataset for k-means to fit
        start : int. Starting range of kmeans to test
        end : int. Ending range of kmeans to test
    OUTPUT:
        Values and line plot of Silhouette Score.
    '''
    
    # Create empty lists to store values for plotting graphs
    n_clu = []
    km_ss = []

    # Create a for loop to find optimal n_clusters
    for n_clusters in range(start, end):

        # Create cluster labels
        kmeans = KMeans(n_clusters=n_clusters)
        labels = kmeans.fit_predict(dataset)

        # Calcualte model performance
        silhouette_avg = round(silhouette_score(dataset, labels, 
                                                random_state=1), 3)

        # Append score to lists
        km_ss.append(silhouette_avg)
        n_clu.append(n_clusters)

        print("No. Clusters: {}, Silhouette Score: {}, Change from Previous Cluster: {}".format(
            n_clusters, 
            silhouette_avg, 
            (km_ss[n_clusters - start] - km_ss[n_clusters - start - 1]).round(3)))

        # Plot graph at the end of loop
        if n_clusters == end - 1:
            plt.figure(figsize=(5.6,3.5))

            #plt.title('Silhouette Score Elbow for KMeans Clustering')
            plt.xlabel('k')
            plt.ylabel('silhouette score')
            sns.pointplot(x=n_clu, y=km_ss)
            plt.savefig('silhouette_score.pdf', format='pdf',
                        pad_inches=2.0)
            plt.tight_layout()
            plt.show()
            

In [ ]:
            
def kmeans(df, clusters_number):
    '''
    Implement k-means clustering on dataset
    
    INPUT:
        dataset : dataframe. Dataset for k-means to fit.
        clusters_number : int. Number of clusters to form.
        end : int. Ending range of kmeans to test.
    OUTPUT:
        Cluster results and t-SNE visualisation of clusters.
    '''
    x = 25000
    kmeans = KMeans(n_clusters = clusters_number, random_state = 1)
    kmeans.fit(df[:x])
    
    labels = kmeans.predict(df[x:])
    # Extract cluster labels
    cluster_labels = kmeans.labels_
        
    # Create a cluster label column in original dataset
    df_new = df[:x].assign(Cluster = cluster_labels)
    
#     # Initialise TSNE
#     model = TSNE(random_state=1)
#     transformed = model.fit_transform(df)
    
#     # Plot t-SNE
#     plt.title('Flattened Graph of {} Clusters'.format(clusters_number))
#     sns.scatterplot(x=transformed[:,0], y=transformed[:,1], hue=cluster_labels, style=cluster_labels, palette="Set1")
#     plt.savefig('cluster_brain_plot_6_clusters_first_2500.png')
    
    
    return df_new, cluster_labels, labels

In [ ]:
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', labelsize=11)
plt.rc('axes', titlesize=11)
plt.rc('xtick', labelsize=9)
plt.rc('ytick', labelsize=9)

### Import and merge the data

In [ ]:
products = pd.read_csv("../instacart/products.csv")
orders = pd.read_csv("../instacart/orders.csv")
order_products_train = pd.read_csv("../instacart/order_products__train.csv")
order_products_prior = pd.read_csv("../instacart/order_products__prior.csv")
departments = pd.read_csv("../instacart/departments.csv")
aisles = pd.read_csv("../instacart/aisles.csv")

In [ ]:
merge_data = products.merge(order_products_prior, on = 'product_id', how = 'inner')
merge_data = departments.merge(merge_data, on = 'department_id', how = 'inner')
merge_data = orders.merge(merge_data, on = 'order_id', how = 'inner')

#remove some useless info
# merge_data = merge_data.drop(['department','product_name'], axis = 1)

In [ ]:
print( "Number of departments:", departments['department_id'].nunique())
print( "Number of aisles:", aisles['aisle_id'].nunique())
print( "Number of products:", products['product_id'].nunique())
print( "Number of unique users:", merge_data['user_id'].nunique())
print( "Number of unique orders", merge_data['order_id'].nunique())

In [ ]:
print("Departments columns:", departments.columns)
print("Aisles columns:", aisles.columns)
print("Product columns:", products.columns)
print("Order_products:" , order_products_prior.columns)
print("Order:" , orders.columns)

### Define functions to calculate the values

In [ ]:
# returns data of User A
def user_specific_data(user_number):
    
    user_data = merge_data_train[merge_data_train['user_id'] == user_number]
    
    return user_data

# returns data of User A and Item B
def user_product_data(user_number,product_number):
    
    user_data = merge_data[merge_data['user_id'] == user_number]
    user_product_data = user_data[user_data['product_id'] == product_number]
    
    
    return user_product_data

#creating crosstabs that indicates the items purchased during each transaction also giving the days since prior-order.
#Visually easy to see which item where purchased in a transaction.
def crosstab_user(user_number):
    
    user_data = user_specific_data(user_number)
    seq = user_data.order_id.unique()
    crosst_user = pd.crosstab(user_data.product_name,user_data.order_id).reindex(seq, axis = 'columns')
    sns.heatmap(crosst_user,cmap="YlGnBu",annot=True, cbar=False)
    
    return crosst_user


def crosstab_user_order_id(user_number):
    
    user_data = user_specific_data(user_number)
    user_data = user_data.fillna(value = 0, axis = 1)
    seq = user_data.order_id.unique()
    dspo_data = user_data.groupby('order_id', as_index=False)['days_since_prior_order'].mean()
    #dspo_data = dspo_data.T
    #user_data = pd.concat([dspo_data,user_data])
        
    crosst_user = pd.crosstab(user_data.product_name,user_data.order_id).reindex(seq, axis = 'columns')
    #sns.heatmap(crosst_user,cmap="YlGnBu",annot=True, cbar=False)
    
    crosst_user = pd.merge((crosst_user.T), dspo_data, on = 'order_id')
    crosst_user = crosst_user.set_index('order_id')
    crosst_user = crosst_user.T
    #sns.heatmap(crosst_user,cmap="YlGnBu",annot=True, cbar=False)
        
    return crosst_user


In [ ]:
# Frequency being the number of orders placed by a user

# Total number of orders placed by a specific user
order_id_grouped = merge_data.drop(['days_since_prior_order','product_id','product_name','add_to_cart_order','reordered'],axis = 1)
number_of_orders_per_user = order_id_grouped.groupby('user_id').agg(num_orders = pd.NamedAgg(column = 'order_id', aggfunc = 'nunique' ))
number_of_orders_per_user

In [ ]:
# plotting the number of products in each order
#creating a graph displaying the time of the day vs the departments
dep_prod = products.merge(departments, on = 'department_id', how = 'inner')
order_order_prod = orders.merge(order_products_prior, on = 'order_id', how = 'inner')
order_dep_prod = dep_prod.merge(order_order_prod,on = 'product_id', how = 'inner')
order_dep_prod_cleaned = order_dep_prod.drop(['days_since_prior_order','add_to_cart_order','reordered','aisle_id','product_id','product_name','order_id','user_id','eval_set'],axis = 1)

In [ ]:
num_prods = order_dep_prod.groupby("order_id")["add_to_cart_order"].aggregate("max").reset_index()
cnt_srs = num_prods.add_to_cart_order.value_counts()
cnt_srs


In [ ]:
# creating a dataframe that specify the number of products in each order for each user

num_prods_user = orders.merge(num_prods, on = 'order_id', how = 'inner')
num_prods_user.drop(['eval_set','order_dow','order_hour_of_day','days_since_prior_order','order_number'],axis = 1)

In [ ]:
# We want the average products per order per user for the monetary entry of RFM
average_num_prods_user =num_prods_user.groupby("user_id")["add_to_cart_order"].aggregate("mean").reset_index()

In [ ]:
#creating a dataframe that contains the Frequency en the monatory values

F_M = number_of_orders_per_user.merge(average_num_prods_user, on = 'user_id', how = 'inner')
F_M = F_M.rename(columns={"num_orders": "Frequency", "add_to_cart_order": "Monetary"})

In [ ]:
#creating the Recency feature
# getting the last days_since_prior_order in the train set....

# using the 2nd last days_since_prior_order as recency
last_days_since_prior_order_user =orders.groupby("user_id")["days_since_prior_order"].nth(-2).reset_index()



# using the average days_since_prior_order as the recency feature
mean_days_since_prior_order_user =orders.groupby("user_id")["days_since_prior_order"].mean().reset_index()





In [ ]:
R_F_M = F_M.merge(mean_days_since_prior_order_user, on = 'user_id', how = 'inner')
RFM = R_F_M.rename(columns={"days_since_prior_order": "Recency"})

In [ ]:
RFM.set_index('user_id', inplace = True)

In [ ]:
#changing the columns so that the order of the columns are RFM 
cols = ['Recency', 'Frequency', 'Monetary']

RFM = RFM[cols]

In [ ]:
RFM

### Checking if the data created is skewed...


In [ ]:
RFM = pd.read_pickle("RFM.pkl")

In [ ]:
#plotting the data to see if the features that we created is skewed
plt.figure(figsize=[5.6,5.6])
RFM.hist(figsize=[5.6,4])
plt.tight_layout()
plt.savefig("RFM.pdf")

From the features we see that the Monetary feature that was created is positively skewed. 

This means that we will have to transform the current data to the log form of the data. 

The orthers are roughly normal, so that we will use it as is...

In [ ]:
#From the figures we see that Frequency (total number of orders per customer) is positively skewed
#thus we need to log transform the data so that we can use K-Means clustering 

RFM['Frequency'] = np.log(RFM['Frequency'])
#RFM['Recency'] = np.log(RFM['Recency'])
#RFM['Monerary'] = np.log(RFM['Monerary'])


# RFM.hist(figsize=(10,6))
# plt.tight_layout()



In [ ]:
RFM['Monetary'] = np.log(RFM['Monetary'])

In [ ]:
df = RFM.drop(['Recency'],axis = 1)

In [ ]:
df.hist(figsize=[5.6,2])
plt.tight_layout()
plt.savefig("rfm_scaled.pdf")

Now the data looks more normal so we will use it as created... 

The data should also be scaled...

In [ ]:
#So now that the data is roughly normal we need to scale the features, because K-Means wants normal data
#around a mean of 0 and a std of 1

#Scaling the RFM features that we created
#This is part of the pre-processing process...
scaling_fact = StandardScaler()
RFM_scaled = scaling_fact.fit_transform(RFM)

RFM_scaled = pd.DataFrame(RFM_scaled)
RFM_scaled.hist(figsize=(10,6))
plt.tight_layout()

In [ ]:
data_described = RFM_scaled.describe()
data_described = data_described.round(decimals=2)
data_described

# Market Segmentation
### Using K-Means to cluster into segments after engineering RFM features

Looking into how many clusters are a good number for this dataset

K-Means performs best when not skewed and when normalised around a mean of 0 and a standard deviation of 1 -- we just did these so we are good to go!

In [ ]:
# Visualize performance of KMeans at various values k
# This approaches uses distortion score to evaluate KMeans
model = KMeans()
plt.figure(figsize= [5.6,3])
visualizer = KElbowVisualizer(model, k=(2, 15))

visualizer.fit(RFM_scaled)   
# plt.tight_layout()
# 
visualizer.show(outpath = "elbow.pdf")
# plt.savefig('elbow.pdf')

In [ ]:
visualizer.show?

In [ ]:
plt.gca().set_xlabel("k")
plt.gca().set_ylabel("distortion score")


In [ ]:
visualizer.savefig('elbow.pdf')

In [ ]:
visualizer.fit(RFM_scaled) 

### With the elbow method it is clear that the number of clusters should be 6

In [ ]:
# Plot clusters for k=3
cluster_less_6, cluster_labels, labels = kmeans(RFM_scaled, 6)

In [ ]:
print(labels.shape)
print(cluster_labels.shape)

In [ ]:
# clusters_3 = kmeans(RFM_scaled, 3)

In [ ]:
# Convert clusters to DataFrame with appropriate index and column names
cluster_df = pd.DataFrame(cluster_less_6)


cluster_df.index = RFM[:25000].index
cluster_df.columns = ['Recency', 'Monetary', 
                      'Frequency', 'Cluster']

In [ ]:
cluster_df

In [ ]:
cluster_df.index.names = ['user_id']
cluster_df.head()

In [ ]:
# Reshape data for snake plot
cluster_melt = pd.melt(cluster_df.reset_index(),
                       id_vars=['user_id', 'Cluster'],
                       value_vars=['Recency',
                                   'Frequency',
                                   'Monetary'],
                       var_name='Metric',
                       value_name='Value')


In [ ]:
cluster_melt['Cluster'] += 1

In [ ]:
# Create snake plot
# palette = ['powderblue', 'green','orange','purple','steelblue','grey']
palette = 'colorblind'
plt.figure(figsize=[5.6,3])
sns.pointplot(x='Metric', y='Value', data=cluster_melt, hue='Cluster', 
              palette=palette)
plt.xlabel('')
plt.ylabel('Value')
plt.yticks([])
#plt.title('Six Customer Segments')
sns.despine()
plt.tight_layout()
# plt.savefig('snake_plot_5_clusters_less_data_head_25000_Av_R2.png', dpi=300, pad_inches=2.0)
plt.savefig('snake_plot_5_clusters_less_data_head_25000_Av_R2.pdf')
plt.show()

### Visualise the clusters

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:

threeD = plt.figure(figsize=[5.6,3]).gca(projection= '3d')
threeD.scatter(cluster_df['Recency'],cluster_df['Frequency'],
               cluster_df['Monetary'], 
               c = cluster_df['Cluster'],cmap='icefire_r')
threeD.set_xlabel('Recency')
threeD.set_ylabel('Frequency')
threeD.set_zlabel('Monetary')

plt.legend()
plt.tight_layout()
plt.savefig('threeD_cluster.png', dpi = 500)

### Format the cluster data into a table

In [ ]:
df_clusters_all_data = pd.DataFrame(cluster_labels)
df_clusters_add = pd.DataFrame(labels)
df_clusters_all_data = df_clusters_all_data.append(df_clusters_add).reset_index()
df_clusters_all = df_clusters_all_data.drop(['index'],axis = 1)
df_clusters_all = df_clusters_all.rename(columns={0:'Cluster'})
df_clusters_all.to_csv('clustered_data.csv')

In [ ]:
#Clustering the customers based on their RFM values
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale

import sklearn.metrics as ss
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
X = scale(RFM)

clustering = KMeans(n_clusters = 5, random_state = 5)
clustering.fit(X)

### Plotting the model output

In [ ]:
%matplotlib inline

In [ ]:
color_theme = np.array(['darkgray','lightsalmon','powderblue','green','yellow'])

In [ ]:
plt.scatter(x = RFM.Frequency, y = RFM.Recency, c = color_theme[clustering.labels_])
plt.title('K-Means classification')

In [ ]:
plt.scatter(x = RFM.Monerary, y = RFM.Recency, c = color_theme[clustering.labels_])
plt.title('K-Means classification')

In [ ]:
plt.scatter(x = RFM.Monerary, y = RFM.Frequency, c = clustering.labels_)
plt.title('K-Means classification')

In [ ]:
def bench_k_means(estimator, name, data):
    estimator.fit(data)
    print('%-9s\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, estimator.inertia_,
             metrics.homogeneity_score(y, estimator.labels_),
             metrics.completeness_score(y, estimator.labels_),
             metrics.v_measure_score(y, estimator.labels_),
             metrics.adjusted_rand_score(y, estimator.labels_),
             metrics.adjusted_mutual_info_score(y,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean')))